## Loading Images

SAM's predictor.set_image function expects the following format for the image:

Arguments:
      image (np.ndarray): The image for calculating masks. Expects an
        image in HWC uint8 format, with pixel values in [0, 255].
      image_format (str): The color format of the image, in ['RGB', 'BGR'].

In [10]:
# load png image into numpy array
import numpy as np
from PIL import Image

path = "/Users/janulm/Documents/ETH/SM10/Computational Intelligence Lab/eth-cil-road-segmentation/data/training/images/satimage_0.png"

im_frame = Image.open(path)
im_frame = im_frame.convert('RGB')
np_frame = np.array(im_frame)


In [11]:
print(np_frame.shape, np_frame.dtype, np_frame.min(), np_frame.max())

(400, 400, 3) uint8 15 252


## Prompt

In [13]:
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator

model_path = "/Users/janulm/Documents/ETH/SM10/Computational Intelligence Lab/eth-cil-road-segmentation/models/sam_vit_h_4b8939.pth"
sam = sam_model_registry["default"](checkpoint=model_path)




# all masks
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(np_frame)



# prompt-based
#predictor = SamPredictor(sam)
#predictor.set_image(np_frame)
#masks, _, _ = predictor.predict(<input_prompts>)

In [14]:
print(masks)

[{'segmentation': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), 'area': 41304, 'bbox': [148, 0, 251, 393], 'predicted_iou': 1.013520359992981, 'point_coords': [[268.75, 143.75]], 'stability_score': 0.9729146957397461, 'crop_box': [0, 0, 400, 400]}, {'segmentation': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), 'area': 14398, 'bbox': [133, 254, 215, 145], 'predicted_iou': 1.0113439559936523

In [12]:
print(help(predictor.set_image))
print(help(predictor.predict))


Help on method set_image in module segment_anything.predictor:

set_image(image: numpy.ndarray, image_format: str = 'RGB') -> None method of segment_anything.predictor.SamPredictor instance
    Calculates the image embeddings for the provided image, allowing
    masks to be predicted with the 'predict' method.
    
    Arguments:
      image (np.ndarray): The image for calculating masks. Expects an
        image in HWC uint8 format, with pixel values in [0, 255].
      image_format (str): The color format of the image, in ['RGB', 'BGR'].

None
Help on method predict in module segment_anything.predictor:

predict(point_coords: Optional[numpy.ndarray] = None, point_labels: Optional[numpy.ndarray] = None, box: Optional[numpy.ndarray] = None, mask_input: Optional[numpy.ndarray] = None, multimask_output: bool = True, return_logits: bool = False) -> Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray] method of segment_anything.predictor.SamPredictor instance
    Predict masks for the given in

## All masks

In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["<model_type>"](checkpoint="<path/to/checkpoint>")
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(<your_image>)